In [1]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import cv2
from os import listdir
from os.path import join, isfile

label_folder = []
total_size = 0
data_path = 'D:\\mnist_testData\\Training_data'

for root, dirts, files in os.walk(data_path):
    # root 為data_path裡面的每個根,
    # dirts為每個根底下的folder,
    # files為每個根底下的檔案(不含根底下的folder中的檔案)
    for dirt in dirts:
        label_folder.append(dirt)
    total_size += len(files)

print('found {} files.'.format(total_size))
print('folder:', label_folder)


found 0 files.
folder: []


In [66]:
# 訓練資料存放進 list
base_x_train = []
base_y_train = []

# 將每個label_folder裡的資料夾裡的檔案 放進 list
for i in range(len(label_folder)):
    label_path = data_path + '\\' + label_folder[i]

    FileName = [f for f in listdir(label_path) if isfile(join(label_path, f))]

    for j in range(len(FileName)):
        path = label_path + '\\' + FileName[j]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        base_x_train.append(img)
        base_y_train.append(label_folder[i])

print(np.array(base_x_train).shape)
print(np.array(base_y_train).shape)

(20000, 28, 28)
(20000,)


In [67]:
# 將 Training_data 分為train, test ，並且轉為矩陣格式(要套進模型的資料必須為np.array)
from sklearn.model_selection import train_test_split
train_images, test_images, train_labels, test_labels = train_test_split( \
    np.array(base_x_train), np.array(base_y_train), test_size=0.2, random_state=0)

In [68]:
train_images = train_images / 255.0
test_images = test_images / 255.0

# input_mat 設定
train_size = train_images.shape[0]
test_size = test_images.shape[0]
input_mat = np.reshape(train_images, (train_size, train_images.shape[1]*train_images.shape[2]))
# test_images = np.reshape(test_images, (10000, 784))
input_mat.shape
train_size

16000

In [69]:
# ymat 設定，改為label coding

ymat = np.repeat(0, 5*train_size).reshape(train_size, 5)
lb = ['2', '3', '4', '5', '7']
for i in range(train_size):
    #ymat[i, int(train_labels[i])] = 1
    ymat[i, lb.index(train_labels[i])] = 1


In [70]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

random.seed(0)

# 參數設定
learning_rate = 0.2

# 每一層neuron的數量
n_in = input_mat.shape[1]
n_hidden = [40, 40] # 隱藏層兩層 各有?, ?個neuron
n_out = ymat.shape[1]

# 第1層權重初始值
w = np.random.randn(n_hidden[0], n_in)

# 第2層權重初始值
w2 = np.random.randn(n_hidden[1], n_hidden[0])

# 第3層權重初始值
w3 = np.random.randn(n_out, n_hidden[1])

t = 0
output_, y = ymat[0], ymat[0]+1

while True:
    
    input_ = input_mat[t % input_mat.shape[0]] # 讓t一直重複跑每一筆資料
    y = ymat[t % input_mat.shape[0]]

    ### 正向傳播

    # 計算output
    hidden_1 = sigmoid(np.matmul(w, input_))
    hidden_2 = sigmoid(np.matmul(w2, hidden_1))
    output_ = sigmoid(np.matmul(w3, hidden_2))



    ### 誤差反向傳播

    # 第3層權重delta計算
    delta3 = (y - output_) * output_ * (1 - output_)

    # 第2層權重delta計算
    delta2 = hidden_2 * (1 - hidden_2) * np.matmul(delta3, w3)

    # 第1層權重delta計算
    delta1 = hidden_1 * (1 - hidden_1) * np.matmul(delta2, w2)

    # 第3層權重修正
    #w3_old = w3.copy()
    w3 += learning_rate * np.tensordot(delta3, hidden_2, axes = 0)

    # 第2層權重修正
    #w2_old = w2.copy()
    w2 += learning_rate * np.tensordot(delta2, hidden_1, axes = 0)

    # 第1層權重修正
    #w_old = w.copy()
    w += learning_rate * np.tensordot(delta1, input_, axes = 0)

    t += 1

    if t > 500000 or (abs(output_ - y) < 0.000001 ).all():
        print('t=',t)
        break

t= 500001


In [71]:
# 將Training_data 中的 test資料 放進訓練好的模型

test_mat = np.reshape(test_images, (test_size, test_images.shape[1]*test_images.shape[2])).T
test_hidden_1 = sigmoid(np.matmul(w, test_mat))
test_hidden_2 = sigmoid(np.matmul(w2, test_hidden_1))
test_output_ = sigmoid(np.matmul(w3, test_hidden_2))

In [72]:
# 將output改為標籤
output_y = np.zeros(test_size)

for i in range(test_size):
    for j in range(5):
        if test_output_.T[i, j] >= max(test_output_.T[i, ]):
            output_y[i] = j

output_y2 = [int(lb[int(output_y[i])]) for i in range(len(test_labels))]
output_y = output_y2

In [73]:
# 標籤改為 int

test_labels_int = [int(test_labels[i]) for i in range(len(test_labels))]

In [74]:
print('test accuracy: {:.2f}'.format(sum(np.array(test_labels_int) == np.array(output_y))/test_size))

test accuracy: 0.96


In [75]:
# 結果寫入檔案開始

file_name = []
total_size = 0
data_path = 'D:\\mnist_testData\\Testing_data'

for root, dirts, files in os.walk(data_path):
    for file in files:
        file_name.append(file)
        
    total_size += len(files)

print('found {} files.'.format(total_size))


found 5000 files.


In [76]:
base_x_test = []

for i in range(len(file_name)):
    label_path = data_path + '\\' + file_name[i]

    img = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
    base_x_test.append(img)

print(np.array(base_x_test).shape)

(5000, 28, 28)


In [77]:
base_test_images = np.array(base_x_test) / 255.0
base_test_size = base_test_images.shape[0]
test_mat = np.reshape(base_test_images, (base_test_size, base_test_images.shape[1]*base_test_images.shape[2])).T
test_hidden_1 = sigmoid(np.matmul(w, test_mat))
test_hidden_2 = sigmoid(np.matmul(w2, test_hidden_1))
test_output_ = sigmoid(np.matmul(w3, test_hidden_2))

In [78]:
output_y = np.zeros(base_test_size)

for i in range(base_test_size):
    for j in range(5):
        if test_output_.T[i, j] >= max(test_output_.T[i, ]):
            output_y[i] = j

#output_raw = np.round_(test_output_.T)
#class_ = np.arange(10)
#output_y = np.matmul(output_raw, class_)

output_y2 = [int(lb[int(output_y[i])]) for i in range(base_test_size)]
output_y = output_y2
output_y

[4,
 7,
 5,
 2,
 3,
 3,
 5,
 4,
 4,
 7,
 5,
 5,
 2,
 3,
 2,
 7,
 3,
 4,
 3,
 5,
 7,
 5,
 4,
 2,
 3,
 3,
 4,
 4,
 3,
 7,
 2,
 4,
 3,
 4,
 4,
 5,
 7,
 7,
 3,
 5,
 3,
 4,
 3,
 4,
 2,
 4,
 2,
 5,
 2,
 2,
 5,
 4,
 4,
 2,
 7,
 2,
 5,
 5,
 5,
 4,
 2,
 2,
 3,
 5,
 7,
 7,
 5,
 2,
 2,
 4,
 4,
 7,
 5,
 4,
 3,
 2,
 4,
 2,
 7,
 5,
 3,
 5,
 3,
 5,
 4,
 4,
 3,
 2,
 3,
 2,
 2,
 5,
 4,
 4,
 2,
 5,
 3,
 5,
 7,
 5,
 5,
 7,
 2,
 2,
 3,
 2,
 3,
 5,
 2,
 7,
 7,
 5,
 3,
 4,
 2,
 7,
 5,
 7,
 7,
 7,
 4,
 3,
 2,
 5,
 5,
 3,
 2,
 7,
 5,
 5,
 7,
 5,
 7,
 3,
 4,
 5,
 2,
 3,
 7,
 5,
 5,
 5,
 2,
 2,
 5,
 2,
 7,
 5,
 3,
 4,
 2,
 2,
 2,
 3,
 2,
 5,
 5,
 3,
 2,
 5,
 2,
 7,
 2,
 4,
 5,
 2,
 7,
 7,
 7,
 4,
 2,
 2,
 2,
 7,
 3,
 2,
 3,
 3,
 4,
 5,
 7,
 7,
 3,
 4,
 5,
 3,
 5,
 4,
 5,
 4,
 7,
 3,
 4,
 3,
 2,
 3,
 2,
 5,
 4,
 4,
 4,
 7,
 2,
 3,
 7,
 2,
 4,
 5,
 4,
 2,
 5,
 4,
 7,
 3,
 2,
 5,
 4,
 3,
 5,
 4,
 5,
 3,
 5,
 4,
 2,
 4,
 2,
 2,
 7,
 3,
 3,
 2,
 4,
 4,
 3,
 7,
 3,
 2,
 3,
 3,
 7,
 4,
 4,
 5,
 5,
 7,
 2,
 5,
 7,
 7,


In [79]:
for i in range(len(file_name)):
    path = data_path + '\\' + file_name[i]
    txt_name = os.path.splitext(file_name[i].split('.')[0])[0]

    with open('711033123.txt', 'a+') as f:
        f.write(txt_name + ' ' + str(int(output_y[i])) + '\n')